In [1]:
//| echo: false
import java.util.stream.Stream;
String path = "../_version.qmd";
Stream<String> lines = Files.lines(Paths.get(path));
String data = lines.collect(Collectors.joining("\n"));
lines.close();

render(data, "text/markdown")

EvalException: ../_version.qmd


Une application utilisant des conteneurs demande d'en coordonner plusieurs c'est que l'on appelle l'orchestration.
L'outil classique est [kubernetes](https://kubernetes.io/) de Google. Il s'agit d'un outils complexe. Une alternative simple est [docker compose](https://docs.docker.com/compose/).  

Docker compose permet de manipuler une architecture logicielle basée sur des conteneurs en décrivant des `services` dans un fichier YAML (`docker-compose.yml`). Les volumes et réseaux sont aussi gérés.
Docker compose reprend les options des commandes de docker client. Docker compose est un simple plugin de docker client à [installer](https://docs.docker.com/compose/install/).

In [2]:
//| output: false
//| echo: false
%printWithName false
String script="""
docker system prune --all --force""";

IJava.getKernelInstance().getMagics().applyCellMagic("shell",List.of(""),script);    

Deleted Images:


untagged: curlimages/curl:7.88.1


untagged: curlimages/curl@sha256:48318407b8d98e8c7d5bd4741c88e8e1a5442de660b47f63ba656e5c910bc3da


deleted: sha256:5880c2bc66d5c2fdec926f2a8d65fee338d703f99f0d08739d2b6605b3e919de


deleted: sha256:6203975d45eb1557a9409c39cf5c8552ae6bac233dca55d0723fe9a04d692986


...

deleted: sha256:a2724430ad9951e1269873d7d2897516916b1f660e8859dd2b9e44d303588980


deleted: sha256:871b98204213215a8459e0955815854c48cf1e73baeccf0bfc5722264f9913db


deleted: sha256:5d3e392a13a0fdfbf8806cb4a5e4b0a92b5021103a146249d8a2c999f06a9772


Total reclaimed space: 14.42MB


IJava.getKernelInstance().getMag...: 0

## Un exemple simple

Dans l'exemple suivant une application est composée de deux services (1) `db` une base de données relationnelles et (2) `app` une application JPA/REST écrit en Java.

Le service `app` s'appuie sur l'exécution d'une image docker de PostgreSQL en lui fixant des variables d'environnements, des volumes et une configuration réseau. 
Il faut noter que les volumes et les réseaux doivent externes définis globalement (dans le cas d'une utilisation avancée il est possible de les contrôler finement). 



```{.docker-compose include="sample-java/restjpa/Docker-compose.yml" code-line-numbers="true"}
```

In [3]:
//| output: false
//| echo: false
%printWithName false
String script="""
pwd
( cd sample-java/restjpa && docker compose down --remove-orphans )""";

IJava.getKernelInstance().getMagics().applyCellMagic("shell",List.of(""),script);   

/home/jovyan/work/local


zsh:cd:2: no such file or directory: sample-java/restjpa


IJava.getKernelInstance().getMag...: 1

Il est possible de lancer explicitement un service avec la commande `docker compose up` depuis le répertoire qui contient le  l'option `-d` permet de le détacher.

In [4]:
%%shell
cd sample-java/restjpa
docker compose up --quiet-pull --detach db

zsh:cd:1: no such file or directory: sample-java/restjpa


 Container local-db-1  Running


la commande `ls` liste les projets en cours d'exécution.

In [5]:
%%shell
docker compose ls -a

NAME                STATUS              CONFIG FILES


local               running(3)          /home/jovyan/work/local/docker-compose.yml


la commande `ps` liste les conteneurs du projet courant, noter que compose nomme automatiquement le conteneur (`<nom_repertoire_parent>-<service>-<id>`). 
Il est donc simple de lancer plusieurs fois le même projet depuis des répertoires différents ou plusieurs fois le même service sans risque de conflit à condition de ne pas utiliser de bind volumes ni de mapper des ports de l'hôte. 

In [6]:
%%shell
cd sample-java/restjpa
docker compose ps -a

zsh:cd:1: no such file or directory: sample-java/restjpa


NAME               IMAGE                         COMMAND                  SERVICE    CREATED          STATUS                    PORTS


local-db-1         postgres:16-alpine            "docker-entrypoint.s…"   db         16 seconds ago   Up 15 seconds             5432/tcp


local-docker-1     docker:dind                   "dockerd-entrypoint.…"   docker     16 seconds ago   Up 15 seconds             2376/tcp, 0.0.0.0:12375->2375/tcp


local-notebook-1   brunoe/jupyter-java:develop   "tini -g -- start-no…"   notebook   16 seconds ago   Up 15 seconds (healthy)   0.0.0.0:4200->4200/tcp, 0.0.0.0:8888->8888/tcp


On remarque que docker compose prend en charge la création et le nommage des réseaux et des volumes en les préfixant du nom du répertoire parent (le projet).

In [7]:
%%shell
docker network ls

NETWORK ID     NAME                DRIVER    SCOPE


78f97fbccea7   bridge              bridge    local


12cb1be39e39   host                host      local


0232a33c5df7   local_notebooknet   bridge    local


3c7330aaa765   none                null      local


In [8]:
%%shell
docker volume ls

DRIVER    VOLUME NAME


local     a98b59ba53d533bab233fd762a090a1f0ebbfa874890a6d469df5ed90ef5db84


local     e10bc368acdb2112b48bf95fbe6f94f938986f3e049ee113984f5d890a11c2be


local     f3ea98831e351e69af60cc962c4e19f5b5118f373b3bd155b1cdd82e046536ad


local     fa72ee147aedb4ef6ba1f89278d46f17c7351bf31229f1612a7a900710b030e8


local     local_certs-ca


local     local_certs-client


local     local_dbhost-data


L'affichage de log d'un service ou de tous les services se fait la commande `logs` (ici limité à 5 lignes).

In [9]:
%%shell
cd sample-java/restjpa
docker compose logs --tail 5

zsh:cd:1: no such file or directory: sample-java/restjpa


db-1  | 2024-03-05 13:06:08.605 UTC [1] LOG:  listening on IPv4 address "0.0.0.0", port 5432


db-1  | 2024-03-05 13:06:08.605 UTC [1] LOG:  listening on IPv6 address "::", port 5432


db-1  | 2024-03-05 13:06:08.607 UTC [1] LOG:  listening on Unix socket "/var/run/postgresql/.s.PGSQL.5432"


db-1  | 2024-03-05 13:06:08.609 UTC [52] LOG:  database system was shut down at 2024-03-05 13:06:08 UTC


...

docker-1    | time="2024-03-05T13:06:09.882870382Z" level=info msg="Loading containers: done."


docker-1    | time="2024-03-05T13:06:09.890941673Z" level=info msg="Docker daemon" commit=f417435 containerd-snapshotter=false storage-driver=overlay2 version=25.0.3


docker-1    | time="2024-03-05T13:06:09.891286298Z" level=info msg="Daemon has completed initialization"


docker-1    | time="2024-03-05T13:06:09.922259132Z" level=info msg="API listen on [::]:2376"


docker-1    | time="2024-03-05T13:06:09.922260715Z" level=info msg="API listen on /var/run/docker.sock"


On peut stopper, détruite ou relancer un service avec `stop`, `rm` et `restart`. 
La commande `down` détruit tous les containeurs et les réseaux. Attention, l'option `-v` supprime les volumes associés nommés ou non. 

In [10]:
%%shell
cd sample-java/restjpa
docker compose down -v

zsh:cd:1: no such file or directory: sample-java/restjpa


 Container local-db-1  Stopping


 Container local-notebook-1  Stopping


 Container local-db-1  Stopped


 Container local-db-1  Removing


...

 Volume local_certs-ca  Removed


 Volume local_certs-client  Removed


 Network local_notebooknet  Removing


 Volume local_dbhost-data  Removed


 Network local_notebooknet  Removed


L'autre service `app` est une application JPA/REST Java dont l'image docker est produite par `sample-java/restjpa/Dockerfile`.
L'option `build` dans `docker-compose.yml` indique qu'il faut fabriquer l'image à partir du contexte courant (`image` sera alors son tag).
La fabrication de l'image sera automatique au démarrage au besoin elle peut être faite manuellement avec `docker compose build`. 

In [11]:
%%shell
cd sample-java/restjpa
docker compose --progress=quiet build
docker compose up --quiet-pull --detach

zsh:cd:1: no such file or directory: sample-java/restjpa


 Network local_notebooknet  Creating


 Network local_notebooknet  Created


 Volume "local_certs-ca"  Creating


 Volume "local_certs-ca"  Created


...

 Container local-db-1  Starting


 Container local-docker-1  Started


 Container local-notebook-1  Starting


 Container local-db-1  Started


 Container local-notebook-1  Started


Il est possible de contrôler l'ordre d'exécution des conteneurs avec la directive `depends_on` dans `docker-compose.yml` pour indiquer quels services doivent être démarrés avant un service. Dans l'exemple, la base de données doit être démarrée avant l'application.

In [12]:
%%shell

sleep 5
    
docker run --network restjpa_frontend \
           --rm --quiet curlimages/curl:7.88.1 \
               curl --verbose --silent \
                 -H "Accept: application/json" \
                 http://restjpa-app-1:8080/restjpa/persons

docker: Error response from daemon: network restjpa_frontend not found.


### Un exemple avancé

L'exemple ci-dessous présente un exemple un peu plus avancé en ajoutant un reverse proxy (https://traefik.io) pour gérer les points d'entrées de l'application (sécurité, répartition de charges, ...).

```{.dockerfile include="sample-java/restjpa/Docker-compose-proxy.yml" code-line-numbers="true"}
```

La directive `healthcheck` de docker compose (qui existe dans le client) permet d'indique une commande à exécuter pour vérifier l'état de santé du conteneur (s'il fonctionne correctement et pas seulement s'il a démarré). Cela peut être utilisé comme condition pour `depends_on`pour ne pas simple lancer un autre service mais attendre qu'il soit fonctionnel.

In [13]:
%%shell
cd sample-java/restjpa
docker compose --file Docker-compose.yml up --quiet-pull --detach

zsh:cd:1: no such file or directory: sample-java/restjpa


 Container local-docker-1  Running


 Container local-notebook-1  Running


 Container local-db-1  Running
